## Cost - Unit labor cost
## Efficiency - productivity

In [ ]:
import pandas as pd
import altair as alt
import numpy as np

df = pd.read_csv("../data/Processed/efficiency_productivity/efficiency_productivity.csv")

# compute yearly average labor productivity and efficiency(producer)
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df_year = df.groupby('Year', as_index=False)[['Labor productivity','Unit labor costs']].mean()
print(df_year)

    Year  Labor productivity  Unit labor costs
0   2013           102.84175          88.68700
1   2014           102.54325          91.29650
2   2015           100.76275          95.46550
3   2016           100.71300          96.18725
4   2017           100.00850          99.99750
5   2018           100.22675         102.18900
6   2019            97.90625         107.20625
7   2020            97.78950         114.53900
8   2021            99.48350         115.33175
9   2022            98.18900         120.61175
10  2023            98.03075         126.75750
11  2024            98.67175         130.99000


In [111]:
# deal with dataset that contains the number of private establishment each quarter
df_estb = pd.read_csv("../data/Processed/num_private_establishment_count.csv")

# convert quarter data to year data. aggregate method: mean
df_estb['Year'] = df_estb['YearQuarter'].str[:4].astype(int)
df_estb_annual = (df_estb.groupby('Year', as_index=False)['num_private_establishment_count'].mean().rename(columns={'num_private_establishment_count': 'Average_Establishments'}))

#print(df_estb_annual)

df_growth = df_estb_annual.sort_values('Year').copy()
df_growth['GrowthPct'] = (
    df_growth['Average_Establishments']
      .pct_change()      
      .mul(100)         
      .round(2)
)

df_growth['GrowthPct'] = df_growth['GrowthPct'].fillna(0)

print(df_growth[['Year','GrowthPct']])

    Year  GrowthPct
0   2013       0.00
1   2014       0.90
2   2015       0.85
3   2016       0.90
4   2017       0.88
5   2018       1.36
6   2019       1.02
7   2020       0.92
8   2021       2.26
9   2022       3.91
10  2023       3.02
11  2024       2.23


In [ ]:
# process worker perspective(wage_hour; employment)

# define function that transform monthly or quarterly data to yearly data
import pandas as pd
import glob
import os

def to_annual(df: pd.DataFrame) -> pd.DataFrame:

    time_col = df.columns[0]
    value_col = df.columns[1]
 
    df2 = df.copy()
    s = df2[time_col].astype(str)

    if df2[time_col].dtype == object:
     
        if df2[time_col].str.contains(r'Q[1-4]').all():

            quarters = s.str.replace(r'\s*([Qq])\s*([1-4])', r'\1\2', regex=True).str.upper()
          
            df2['Date'] = pd.PeriodIndex(quarters, freq='Q').to_timestamp()
        else:
          
            df2['Date'] = pd.to_datetime(df2[time_col])
    else:
      
        df2['Date'] = pd.to_datetime(df2[time_col])


    df2['Year'] = df2['Date'].dt.year

    annual_df = (
        df2
        .groupby('Year', as_index=False)[value_col]
        .mean()
        .rename(columns={value_col: f'{value_col}_annual_mean'})
    )
    return annual_df

# automatically process employment data
raw_folder = "../data/Processed/selected_data_for_worker/employment"          
pattern    = os.path.join(raw_folder, "*.csv")
files      = glob.glob(pattern)

annual_results = {}  

for path in files:
    name = os.path.splitext(os.path.basename(path))[0]
    df   = pd.read_csv(path)
    try:
        df_annual = to_annual(df)
    except Exception as e:
        print(f" Errors when processing {name}: {e}")
        continue

    annual_results[name] = df_annual

for name, df_ann in annual_results.items(): 
    df_ann.to_csv(f"../data/Processed/selected_data_for_worker/employment/{name}_annual.csv", index=False)

# automatically process wage_hour data
raw_folder = "../data/Processed/selected_data_for_worker/wage_hour"          
pattern    = os.path.join(raw_folder, "*.csv")
files      = glob.glob(pattern)

annual_results = {}  

for path in files:
    name = os.path.splitext(os.path.basename(path))[0]
    df   = pd.read_csv(path)
    try:
        df_annual = to_annual(df)
    except Exception as e:
        print(f" Errors when processing {name}: {e}")
        continue

    annual_results[name] = df_annual

for name, df_ann in annual_results.items(): 
    df_ann.to_csv(f"../data/Processed/selected_data_for_worker/wage_hour/{name}_annual.csv", index=False)

In [117]:
# edit policy bar
import pandas as pd

policies = pd.DataFrame([
    {'start':'2015-01-01', 'end':'2016-06-30', 'policy':'A'},
    {'start':'2016-07-01', 'end':'2018-03-31', 'policy':'B'},
    {'start':'2018-04-01', 'end':'2020-12-31', 'policy':'C'},
])
policies['start'] = pd.to_datetime(policies['start'])
policies['end']   = pd.to_datetime(policies['end'])

In [137]:
# prepare the data for drawing parallel coordinate
import pandas as pd

# employment data
df_employ_annual = pd.read_csv("../data/Processed/selected_data_for_worker/employment/all_employees_thousands_annual.csv")
df_job_open_annual = pd.read_csv("../data/Processed/selected_data_for_worker/employment/job_opennings_thousands_annual.csv")
df_num_privest_jbg_annual = pd.read_csv("../data/Processed/selected_data_for_worker/employment/num_priv_estab_gross_job_gains_thousands_annual.csv")
df_num_privest_jbl_annual = pd.read_csv("../data/Processed/selected_data_for_worker/employment/num_priv_estab_gross_job_losses_thousands_annual.csv")
df_unemploy_rate_annual = pd.read_csv("../data/Processed/selected_data_for_worker/employment/unemployment_rate_annual.csv")

df_employment = (
    df_employ_annual
    .merge(df_job_open_annual,       on='Year', how='outer')
    .merge(df_num_privest_jbg_annual, on='Year', how='outer')
    .merge(df_num_privest_jbl_annual, on='Year', how='outer')
    .merge(df_unemploy_rate_annual,  on='Year', how='outer')
)

# wage_hour data
df_avg_earnings_annual = pd.read_csv("../data/Processed/selected_data_for_worker/wage_hour/avg_hourly_earnings_dollars_annual.csv")
df_avg_work_hours_annual = pd.read_csv("../data/Processed/selected_data_for_worker/wage_hour/avg_weekly_work_hours_annual.csv")
df_compensation_hour_annual = pd.read_csv("../data/Processed/selected_data_for_worker/wage_hour/priv_total_compensation_cost_dollars_per_hour_annual.csv")
df_salary_hour_annual = pd.read_csv("../data/Processed/selected_data_for_worker/wage_hour/priv_wage_salaries_cost_dollars_per_hour_annual.csv")

df_wage_hour = (
    df_avg_earnings_annual 
    .merge(df_avg_work_hours_annual,       on='Year', how='outer')
    .merge(df_compensation_hour_annual, on='Year', how='outer')
    .merge(df_salary_hour_annual,  on='Year', how='outer')
)

print(df_wage_hour.tail())


    Year  avg_hourly_earnings_dollars_annual_mean  \
7   2020                                28.753333   
8   2021                                29.663333   
9   2022                                30.936667   
10  2023                                32.395833   
11  2024                                33.972500   

    avg_weekly_work_hours_annual_mean  \
7                           39.841667   
8                           40.416667   
9                           40.358333   
10                          40.083333   
11                          40.041667   

    priv_total_compensation_cost_dollars_per_hour_Estimate Value_annual_mean  \
7                                             39.8300                          
8                                             40.8675                          
9                                             42.5450                          
10                                            43.7200                          
11                                

In [ ]:
# visualization test parallel coordinate
import pandas as pd
import altair as alt

df_long = df_wage_hour.melt(
    id_vars=['Year'],
    value_vars=[
        'avg_hourly_earnings_dollars_annual_mean',
        'avg_weekly_work_hours_annual_mean',
        'priv_total_compensation_cost_dollars_per_hour_Estimate Value_annual_mean',
        'priv_wage_salaries_cost_dollars_per_hour_Estimate Value_annual_mean'
    ],
    var_name='Metric',
    value_name='Value'
)

label_map = {
    'avg_hourly_earnings_dollars_annual_mean': 'Hourly Earnings',
    'avg_weekly_work_hours_annual_mean':    'Weekly Hours',
    'priv_total_compensation_cost_dollars_per_hour_Estimate Value_annual_mean': 'Total Comp/hr',
    'priv_wage_salaries_cost_dollars_per_hour_Estimate Value_annual_mean':    'Wage Cost/hr'
}
df_long['Metric'] = df_long['Metric'].map(label_map)

base = alt.Chart(df_long).encode(
    x=alt.X('Metric:O', title=None, axis=alt.Axis(orient='top', labelAngle=0, grid=True)),
    y=alt.Y('Value:Q', title=None, scale=alt.Scale(domain=[20,50])),
    detail='Year:O',        
    color='Year:O',     
    tooltip=['Year:O','Metric:N','Value:Q']
)

lines  = base.mark_line(strokeWidth=2)
points = base.mark_point(size=60, filled=True)

parallel_coords = (lines + points).properties(
    width=800, height=300,
    title='Annual Worker Metrics Parallel Coordinates'
)

parallel_coords

alt.LayerChart(...)

In [ ]:
# import pandas as pd
# import altair as alt
# import numpy as np

# years = np.arange(2013, 2025)
# np.random.seed(42)
# df = pd.DataFrame({
#     'Year': np.repeat(years, 1),
#     'Hourly Earnings':   25 + np.random.randn(len(years))*2,
#     'Total Comp':        35 + np.random.randn(len(years))*3,
#     'Wage Cost':         22 + np.random.randn(len(years))*1.5,
#     'Weekly Hours':      40 + np.random.randn(len(years))*1
# })

# units = {
#     'Hourly Earnings': 'USD/hr',
#     'Total Comp':      'USD/hr',
#     'Wage Cost':       'USD/hr',
#     'Weekly Hours':    'hours'
# }
# df_long = df.melt(id_vars='Year', var_name='Metric', value_name='Value')
# df_long['MetricLabel'] = df_long.apply(
#     lambda row: f"{row.Metric}\n({units[row.Metric]})", axis=1
# )

# brush = alt.selection_interval(encodings=['x'])

# charts = []
# for mlabel in df_long['MetricLabel'].unique():
#     c = (
#         alt.Chart(df_long)
#          .transform_filter(f"datum.MetricLabel === '{mlabel}'")
#          .mark_line(interpolate='monotone', strokeWidth=2)
#          .encode(
#              x=alt.X('MetricLabel:N',
#                      axis=alt.Axis(labelAngle=0, title=None)),
#              y=alt.Y('Value:Q',
#                      axis=alt.Axis(title=None)),
#              color=alt.condition(brush,
#                                  'Year:O',
#                                  alt.value('lightgray'),
#                                  legend=None),
#              opacity=alt.condition(brush,
#                                   alt.value(1),
#                                   alt.value(0.2))
#          )
#          .add_parameter(brush)
#     )
#     charts.append(c)


# parallel = alt.hconcat(*charts, spacing=0).resolve_scale(
#     y='independent'
# ).properties(
#     title='Annual Worker Metrics Parallel Coordinates',
# )

# parallel


AttributeError: 'Chart' object has no attribute 'add_parameter'

In [ ]:
# visualization

# build policy bar
policy_bars = alt.Chart(policies).mark_rect().encode(
    x='start:T',
    x2='end:T',
    y=alt.value(0),        
    y2=alt.value(1),
    color=alt.Color('policy:N',
                    scale=alt.Scale(scheme='category10'))
).properties(height=30)

policy_labels = alt.Chart(policies).mark_text(
    baseline='middle', dy=-10, color='black'
).encode(
    x=alt.X('start:T', title=None),   
    text='policy:N'
).properties(height=30)

policy_row = alt.layer(policy_bars, policy_labels).properties(
    width=800,   
    title='Policy'
)

# build the slider
year_slider = alt.param(
    name='year_sel',
    bind=alt.binding_range(
        name='Show up to Year: ',
        min=int(df_year['Year'].min()),
        max=int(df_year['Year'].max()),
        step=1
    ),
    value=int(df_year['Year'].min())
)

# build the connected scatter plot
base = alt.Chart(df_year).transform_filter("datum.Year <= year_sel").encode(
    x=alt.X('Labor productivity:Q', title='Labor Productivity (Index, 2017=100)', scale=alt.Scale(domain=[97.5, 103])),
    y=alt.Y('Unit labor costs:Q',  title='Unit Labor Costs (Index, 2017=100)', scale=alt.Scale(domain=[80, 140])),
    order=alt.Order('Year:O'),
    tooltip=[
      alt.Tooltip('Year:O', title='Year'),
      alt.Tooltip('Labor productivity:Q', format='.2f'),
      alt.Tooltip('Unit labor costs:Q', format='.2f'),
    ]
)

line   = base.mark_line(color='#1f77b4', strokeWidth=1)
points = base.mark_point(color='#ff7f0e', size=100, filled=True)

scatter = (line + points).properties(
    width=800,    
    height=200
)

# build the bar plot
bar = (
    alt.Chart(df_estb_annual)
       .transform_filter("datum.Year == year_sel")
       .mark_bar(color='teal')
       .encode(
           y=alt.Y('Year:O', title=None, sort='descending'),
           x=alt.X('Average_Establishments:Q', title='Avg Private Establishments', scale = alt.Scale(domain=[0,420000])),
       )
).properties(
    width=800,   
    height=20
)

# build the line chart
years = list(range(2013, 2025))

base_growth = (
    alt.Chart(df_growth)
       .transform_filter("datum.Year <= year_sel")
       .encode(
           x=alt.X('Year:O', title='Year', scale=alt.Scale(domain=years)),
           y=alt.Y('GrowthPct:Q', title='Annual Growth Rate (%)', scale=alt.Scale(domain=[0, 4])),
           tooltip=[
               alt.Tooltip('Year:O'),
               alt.Tooltip('GrowthPct:Q', title='Growth Rate (%)')
           ]
       )
)

line_growth = base_growth.mark_line(color='darkgreen', strokeWidth=2)
points_growth = base_growth.mark_point(color='darkgreen', size=80)

growth_chart = (
    (line_growth + points_growth)
    .properties(
        height=100,
        width=800,
        title='Private Establishment Annual Growth Rate'
    )
)

# assemble
chart = alt.vconcat(
    policy_row,
    scatter,
    bar,
    growth_chart
).add_params(
    year_slider
).properties(
    title='Linked Connected Scatter & Annual Bar Chart'
).configure_title(    
        anchor='middle'
    )

chart

alt.VConcatChart(...)